In [ ]:
from langgraph.graph import StateGraph, END
from modules.agent_state import AgentState
from modules.nodes import Nodes

def build_stategraph():
    builder = StateGraph(AgentState)
    
    # Adding nodes
    builder.add_node("user_input", Nodes.user_input_node)
    builder.add_node("retrieve", Nodes.retrieve_node)
    builder.add_node("generate_response", Nodes.generate_response_node)
    builder.add_node("update_memory", Nodes.update_memory_node)
    
    # Defining flow
    builder.set_entry_point("user_input")
    builder.add_edge("user_input", "retrieve")
    builder.add_edge("retrieve", "generate_response")
    builder.add_edge("generate_response", "update_memory")
    builder.add_edge("update_memory", END)
    
    return builder.compile()

def main():
    rag_graph = build_stategraph()
    
    while True:
        query = input("Enter your query: ")
        if query.lower() in ["exit", "quit"]:
            break
        
        for step in rag_graph.stream({"user_query": query}):
            print("Response:", step)

if __name__ == "__main__":
    main()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
from langgraph.graph import StateGraph, END
from modules.agent_state import AgentState
from modules.nodes import Nodes

def build_stategraph():
    builder = StateGraph(AgentState)
    
    # Adding nodes
    builder.add_node("user_input", Nodes.user_input_node)
    builder.add_node("retrieve", Nodes.retrieve_node)
    builder.add_node("generate_response", Nodes.generate_response_node)
    builder.add_node("update_memory", Nodes.update_memory_node)
    
    # Defining flow
    builder.set_entry_point("user_input")
    builder.add_edge("user_input", "retrieve")
    builder.add_edge("retrieve", "generate_response")
    builder.add_edge("generate_response", "update_memory")
    builder.add_edge("update_memory", END)
    
    return builder.compile()

import gradio as gr

# Build the stategraph only once
rag_graph = build_stategraph()

def process_query1(query):
    # Collect responses from the stategraph and join them in a single string.
    responses = []
    for step in rag_graph.stream({"user_query": query}):
        responses.append(str(step))
    return "\n".join(responses)

def process_query(query):
    # Collect responses from the stategraph
    last_response = None
    for step in rag_graph.stream({"user_query": query}):
        last_response = step  # Keep updating to get the last response
    
    # Convert the last response to string if it's not already
    return str(last_response) if last_response else "No response generated."


# Create the Gradio interface.
iface = gr.Interface(
    fn=process_query,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs="text",
    title="RAG Graph Interface",
    description="Enter a query and get a response from the stategraph."
)

# Launch Gradio; for notebooks, share=False usually suffices.
iface.launch()


Running on local URL:  http://127.0.0.1:7868

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


In [19]:
import gradio as gr
from langgraph.graph import StateGraph, END
from modules.agent_state import AgentState
from modules.nodes import Nodes

def build_stategraph():
    builder = StateGraph(AgentState)
    
    # Adding nodes
    builder.add_node("user_input", Nodes.user_input_node)
    builder.add_node("retrieve", Nodes.retrieve_node)
    builder.add_node("generate_response", Nodes.generate_response_node)
    builder.add_node("update_memory", Nodes.update_memory_node)
    
    # Defining flow
    builder.set_entry_point("user_input")
    builder.add_edge("user_input", "retrieve")
    builder.add_edge("retrieve", "generate_response")
    builder.add_edge("generate_response", "update_memory")
    builder.add_edge("update_memory", END)
    
    return builder.compile()

# Create the StateGraph
rag_graph = build_stategraph()

def respond(message, history):
    # Run the graph and get the final state
    final_state = rag_graph.invoke({"user_query": message})
    
    # Extract the AI's response from the final state
    # Adjust this based on your AgentState structure
    ai_response = final_state.get("response", "I don't have an answer for that.")
    
    return ai_response

# Create and launch the Gradio interface
demo = gr.ChatInterface(
    fn=respond,
    title="LangGraph RAG Assistant",
    description="Ask me anything and I'll use a retrieval-augmented generation system to answer.",
    examples=["Tell me about machine learning", "What is LangGraph?", "How does RAG work?"],
    theme=gr.themes.Soft()
)


demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7875

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/02/25 20:10:58 [W] [service.go:132] login to server failed: i/o deadline reached


In [5]:
import gradio as gr
import os
import warnings
import shutil
from datetime import datetime
from typing import List
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Import the shared state and nodes modules
from modules.agent_state import AgentState
from modules.nodes import Nodes

# Environment setup
load_dotenv()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')

# Directory paths
VECTOR_STORE_PATH = "vector-db"
UPLOADED_FILES_DIR = "uploaded_files"

# Ensure directories exist
os.makedirs(VECTOR_STORE_PATH, exist_ok=True)
os.makedirs(UPLOADED_FILES_DIR, exist_ok=True)

class ServerNodes:
    """Server-specific nodes for the StateGraph"""
    
    @staticmethod
    def load_pdf_node(state):
        """Node to load and process PDF files"""
        pdf_files = state.get("pdf_files", [])
        if not pdf_files:
            return {"status": "No PDF files provided", **state}
        
        all_documents = []
        # Load each PDF
        for pdf_file in pdf_files:
            try:
                loader = PyPDFLoader(pdf_file)
                documents = loader.load()
                all_documents.extend(documents)
            except Exception as e:
                return {
                    "status": f"Error loading PDF {pdf_file}: {str(e)}", 
                    # Add a placeholder response to satisfy the state requirements
                    "response": f"Error loading PDF: {str(e)}",
                    **state
                }
        
        # Split documents
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=100,
            separators=["\n\n", "\n", " ", ""]
        )
        split_docs = text_splitter.split_documents(all_documents)
        
        return {
            "status": f"Successfully loaded {len(pdf_files)} PDF(s) with {len(split_docs)} chunks",
            "documents": split_docs,
            # Add a placeholder response to satisfy the state requirements
            "response": f"Successfully loaded {len(pdf_files)} PDF(s) with {len(split_docs)} chunks",
            **state
        }
    
    @staticmethod
    def update_vector_store_node(state):
        """Node to update the vector store with new documents"""
        documents = state.get("documents", [])
        if not documents:
            return {
                "status": "No documents to add to vector store", 
                # Add a placeholder response to satisfy the state requirements
                "response": "No documents to add to vector store",
                **state
            }
        
        try:
            # Initialize embeddings
            embeddings = OllamaEmbeddings(model="nomic-embed-text") 
            
            # Check if vector store exists
            if os.path.exists(VECTOR_STORE_PATH):
                # Load existing vector store
                vector_store = Chroma(persist_directory=VECTOR_STORE_PATH, embedding_function=embeddings)
                # Add new documents
                vector_store.add_documents(documents)
            else:
                # Create new vector store
                vector_store = Chroma.from_documents(
                    documents=documents,
                    embedding=embeddings,
                    persist_directory=VECTOR_STORE_PATH
                )
            
            # Persist changes
            vector_store.persist()
            
            success_message = f"Successfully updated vector store with {len(documents)} documents"
            return {
                "status": success_message,
                "vector_store": vector_store,
                # Add a placeholder response to satisfy the state requirements
                "response": success_message,
                **state
            }
        except Exception as e:
            error_message = f"Error updating vector store: {str(e)}"
            return {
                "status": error_message,
                # Add a placeholder response to satisfy the state requirements
                "response": error_message,
                **state
            }

def build_server_stategraph():
    """Build the StateGraph for the server"""
    builder = StateGraph(AgentState)
    
    # Add server-specific nodes
    builder.add_node("load_pdf", ServerNodes.load_pdf_node)
    builder.add_node("update_vector_store", ServerNodes.update_vector_store_node)
    
    # Add shared nodes (for chat functionality)
    builder.add_node("user_input", Nodes.user_input_node)
    builder.add_node("retrieve", Nodes.retrieve_node)
    builder.add_node("generate_response", Nodes.generate_response_node)
    builder.add_node("update_memory", Nodes.update_memory_node)
    
    # Define PDF processing flow
    builder.add_conditional_edges(
        "user_input",
        lambda state: "load_pdf" if state.get("pdf_files") else "retrieve"
    )
    builder.add_edge("load_pdf", "update_vector_store")
    builder.add_edge("update_vector_store", END)
    
    # Define chat flow
    builder.add_edge("retrieve", "generate_response")
    builder.add_edge("generate_response", "update_memory")
    builder.add_edge("update_memory", END)
    
    # Set entry point
    builder.set_entry_point("user_input")
    
    return builder.compile()

# Create a separate graph for PDF processing to avoid state validation issues
def build_pdf_processing_graph():
    """Build a separate StateGraph just for PDF processing"""
    # Create a custom state class for PDF processing that doesn't require the same fields
    class PDFProcessState(dict):
        pass
    
    builder = StateGraph(PDFProcessState)
    
    # Add PDF processing nodes
    builder.add_node("load_pdf", ServerNodes.load_pdf_node)
    builder.add_node("update_vector_store", ServerNodes.update_vector_store_node)
    
    # Define flow
    builder.set_entry_point("load_pdf")
    builder.add_edge("load_pdf", "update_vector_store")
    builder.add_edge("update_vector_store", END)
    
    return builder.compile()

# Create the StateGraphs
server_graph = build_server_stategraph()
pdf_process_graph = build_pdf_processing_graph()

def save_uploaded_files(files):
    """Save uploaded files to the designated directory and return their paths"""
    saved_paths = []
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    for i, file in enumerate(files):
        # Create a unique filename with timestamp
        original_filename = os.path.basename(file.name)
        filename_without_ext, extension = os.path.splitext(original_filename)
        new_filename = f"{filename_without_ext}_{timestamp}_{i}{extension}"
        
        # Define the save path
        save_path = os.path.join(UPLOADED_FILES_DIR, new_filename)
        
        # Copy the file to the uploads directory
        shutil.copy2(file.name, save_path)
        saved_paths.append(save_path)
    
    return saved_paths

def list_uploaded_files():
    """List all previously uploaded PDF files"""
    if not os.path.exists(UPLOADED_FILES_DIR):
        return []
    
    files = [f for f in os.listdir(UPLOADED_FILES_DIR) if f.lower().endswith('.pdf')]
    return sorted(files, reverse=True)  # Sort by newest first

def process_pdfs(pdf_files):
    """Process uploaded PDF files and update the vector store"""
    if not pdf_files:
        return "No PDF files uploaded"
    
    # Save the uploaded files to the upload directory
    saved_paths = save_uploaded_files(pdf_files)
    
    try:
        # Use the dedicated PDF processing graph instead of the main server graph
        final_state = pdf_process_graph.invoke({
            "pdf_files": saved_paths
        })
        
        status = final_state.get("status", "Processing completed")
        return f"{status}\nFiles saved to {UPLOADED_FILES_DIR} directory."
    except Exception as e:
        return f"Error processing PDFs: {str(e)}\nFiles were saved to {UPLOADED_FILES_DIR} directory."

def respond(message, history):
    """Handle chat interaction"""
    # Run the graph with user query
    final_state = server_graph.invoke({
        "user_query": message
    })
    
    # Extract the AI's response
    ai_response = final_state.get("response", "I don't have an answer for that.")
    
    return ai_response

# Create Gradio interface with tabs for different functions
with gr.Blocks(title="IZTECH Telecom RAG Server 🤖", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# IZTECH Telecom RAG Server")
    
    with gr.Tabs():
        # PDF Upload Tab
        with gr.TabItem("PDF Upload"):
            with gr.Row():
                pdf_input = gr.File(
                    file_count="multiple", 
                    file_types=['.pdf'], 
                    label="Upload PDF Files"
                )
            
            with gr.Row():
                process_btn = gr.Button("Process PDFs")
                status_output = gr.Textbox(label="Processing Status", interactive=False)
            
            with gr.Row():
                gr.Markdown("### Previously Uploaded Files")
                files_list = gr.Dataframe(
                    headers=["Filename"],
                    datatype=["str"],
                    label="Uploaded PDFs",
                    interactive=False
                )
            
            # Connect process button to process_pdfs function
            process_btn.click(
                fn=process_pdfs,
                inputs=[pdf_input],
                outputs=[status_output]
            ).then(
                fn=lambda: {"Filename": list_uploaded_files()},
                outputs=[files_list]
            )
            
            # Initialize the files list on page load
            demo.load(
                fn=lambda: {"Filename": list_uploaded_files()},
                outputs=[files_list]
            )
        
        # Chat Tab
        with gr.TabItem("Chat"):
            chatbot = gr.ChatInterface(
                fn=respond,
                title="Chat with RAG System",
                description="Ask questions about the uploaded PDFs"
            )

# Launch the application
if __name__ == "__main__":
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7864
Running on public URL: https://f37c2011bc38fbcc79.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
